<a href="https://colab.research.google.com/github/sreeduttsasikumar/Coursera_Capstone/blob/main/Capstone_Project_Battle_of_Neighbourhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Capstone Project - The Battle of the Neighborhoods</center>
# <center> Analyzing major cities in Kerala, India for choosing the site for a Residential Project <center>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

One of the major Real Estate investor in India, based out of Mumbai, **wants to construct a 5-star Residential Complex in any major city in Kerala**. To that purpose, he wants to analyze the neighbourhood of the cities and choose a neighbourhood/city which will be suitable for the project.

He is planning to have a **water facing villa theme**, and the water body can be anything like Beaches, Lakes, River, Backwater, etc. Also, the **locality should be suitable for a 5 star residential villa with all necessary facilities and ameninites nearby**

We will use the power of Data Science to map the requirements described against the data available and find out one/mutiple cities which will be well suited for the project.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* presence of water body in locality
* presense of amentities for household such as Grocery, Schools, Hospitals, etc
* should not be a trending/happening locality as that won't be suitable for a residential area

We decided to use the wikipedia page which lists the major Muncipality and Muncipality Corporations in Kerala. Will extract the data from the table and will use those locations for finding out the perfect location as described in the requirement.

By using the locations details downloaded, corresponding latitudes and longitudes will be captured using geocoder api. Also using foursquare api we will find out the neighbouring venues of those locations which will be the basis for the analysis we do in this project

Lets start the journey of data exploration, data enriching, data cleaning, data transfomration, etc etc

Use below cell to consolidate all the library package importing

In [35]:
#pandas don't need much info. Anyways imported for dataframe functionality
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# map rendering library
import folium
# library to handle JSON files 
import json
# library to handle requests
import requests
#library to handle mathematical equations
import numpy as np
# import k-means from clustering stage
from sklearn.cluster import KMeans

Lets download the data containing the major cities in Kerala from the identified wikipedia page

In [2]:
#Assign the wikipedia url to variable url
url = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Kerala'
#fetch from the url using panda function
data_from_url = pd.read_html(url)
#get the second table which is the relevant table for our project
kerala_city_df = data_from_url[1]
#drop columns not needed for analysis
kerala_city_df.drop(kerala_city_df.columns[[0, 2, 3, 4]], axis = 1, inplace = True) 
#rename columns to meaningful names
kerala_city_df.columns = ['City', 'Muncipality']
#correct a few data for proper address resolution and coordinates fecthing by geocoder
kerala_city_df.City[kerala_city_df.City=='Thiruvananthapuram'] = 'Trivandrum, TVM'
kerala_city_df.Muncipality[kerala_city_df.Muncipality=='Thiruvananthapuram'] = 'Trivandrum'
kerala_city_df.drop(index = kerala_city_df[kerala_city_df['City']=='Municipalities'].index[0], inplace= True)
kerala_city_df.reset_index(inplace= True, drop=True)
print("Total number of cities available for analysis are {}.".format(kerala_city_df.shape[0]))
kerala_city_df.head()

Total number of cities available for analysis are 93.


,City,Muncipality
0,"Trivandrum, TVM",Trivandrum
1,Kozhikode,Kozhikode
2,Kochi,Ernakulam
3,Kollam,Kollam
4,Thrissur,Thrissur


####Lets use Geocoder to fetch the latitude and longitudes of the cities and update them in the dataframe

Let's define below function to convert the city to corresponding latitudes and longitudes

In [3]:
 def lat_lon_from_address(cities):  
  city_lat_long_temp = []
  for city in cities: 
    geolocator = Nominatim(user_agent="city_explorer")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    #print('The geograpical coordinate of {} are {}, {}.'.format(city, latitude, longitude))
    city_lat_long_temp.append([city, latitude, longitude])
  return (city_lat_long_temp)

Call the above defined function by passing the city names as parameters and assign the newly found latitudes and longitudes in a new dataframe. This will be then merged with main dataframe kerala_city_df

In [4]:
#call the function
city_lat_long = pd.DataFrame(lat_lon_from_address(kerala_city_df['City']))
#set columns for the new dataframe
city_lat_long.columns = ['City', 'City Latitude', 'City Longitude']
#using pandas merge function merge the initial data set and the coordinte data set using the common column City
kerala_city_df = pd.merge(kerala_city_df, city_lat_long, left_on='City', right_on='City')
kerala_city_df.head()

,City,Muncipality,City Latitude,City Longitude
0,"Trivandrum, TVM",Trivandrum,8.487803,76.951847
1,Kozhikode,Kozhikode,11.258608,75.778874
2,Kochi,Ernakulam,9.931308,76.267414
3,Kollam,Kollam,8.887951,76.595501
4,Thrissur,Thrissur,10.525626,76.213254


#### Create a map of Kerala with neighborhoods superimposed on top.

In [5]:
#get geo location of Kerala on which the cities can be super imposed
geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode('Kerala, KL')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kerala are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kerala are 9.5797046, 76.5691745.


In [6]:
map_kerala = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, muncipality, city in zip(kerala_city_df['City Latitude'], kerala_city_df['City Longitude'], kerala_city_df['Muncipality'], kerala_city_df['City']):
    label = '{}, {}'.format(city, muncipality)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kerala)  
    
map_kerala

#### Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'UDO03A1SPAJOCS4SJODJ31ORPYBC2NA2M2ND4QLRL51LNO3H' # your Foursquare ID
CLIENT_SECRET = '3LBAEVCJ21YGCIMYN1LCZBXWCHA4CE1QI511MI4HJP4ASSXN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#### Let's create a function to fetch nearby venues around the cities in Kerala

In [8]:
def getNearbyVenues(names, muncipalities, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, muncipality, lat, lng in zip(names, muncipalities, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            muncipality,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Muncipality', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _kerala_venues_.

In [9]:
kerala_venues = getNearbyVenues(names=kerala_city_df['City'],
                                   muncipalities=kerala_city_df['Muncipality'],
                                   latitudes=kerala_city_df['City Latitude'],
                                   longitudes=kerala_city_df['City Longitude']
                                  )
kerala_venues.head()

,City,City Latitude,City Longitude,Muncipality,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Trivandrum, TVM",8.487803,76.951847,Trivandrum,Nila Theater,8.489039,76.953007,Movie Theater
1,"Trivandrum, TVM",8.487803,76.951847,Trivandrum,Kairali Sree Nila Theatre Complex,8.489033,76.953024,Multiplex
2,"Trivandrum, TVM",8.487803,76.951847,Trivandrum,Ariesplex SL Cinemas,8.488652,76.946331,Multiplex
3,"Trivandrum, TVM",8.487803,76.951847,Trivandrum,Hotel High Land,8.488651,76.949176,Arcade
4,"Trivandrum, TVM",8.487803,76.951847,Trivandrum,Appolo Dimora,8.488882,76.950888,Hotel


#### Let's check the size of the resulting dataframe

In [52]:
print("Out of {} cities in the initial transformed dataset, {} cities were populated in the resulting dataset after nearby venue search.".format(kerala_city_df.shape[0], len(kerala_venues['City'].unique())))

Out of 93 cities in the initial transformed dataset, 64 cities were populated in the resulting dataset after nearby venue search.


In [53]:
print("So ineffect {} cities were filtered out as foursquare API couldn't fetch any nearby venues of those cities.".format(kerala_city_df.shape[0] - len(kerala_venues['City'].unique())))

So ineffect 29 cities were filtered out as foursquare API couldn't fetch any nearby venues of those cities.


Below are those cities, which got filtered out. At later stage, will try to correct the initial data, so that these could be also be included the final data set

In [65]:
#create new df without the cities having no venues. Need this for clustering
kerala_city_filtered_df = kerala_city_df[kerala_city_df.City.isin(kerala_venues['City'])].reset_index(drop=True)
#displaying the cities with no venues
kerala_city_df[~kerala_city_df.City.isin(kerala_venues['City'])].reset_index(drop=True)

,City,Muncipality,City Latitude,City Longitude
0,Alappuzha,Alappuzha,9.488707,76.415214
1,Kottayam,Kottayam,9.628738,76.645533
2,Thalassery,Kannur,11.835651,75.644905
3,Ponnani,Malappuram,10.774822,75.979768
4,Malappuram,Malappuram,11.106845,76.109955
5,Panoor,Kannur,11.724799,75.588959
6,Tirur,Malappuram,10.904113,76.002817
7,Changanassery,Kottayam,9.489773,76.627533
8,Kunnamkulam,Thrissur,10.653817,76.099330
9,Kasaragod,Kasaragod,12.421713,75.190450


In [63]:
print("Total {} venues were identified amongst {} available cities from Kerala".format(kerala_venues.shape[0], len(kerala_venues['City'].unique())))

Total 453 venues were identified amongst 64 available cities from Kerala


Let's check how many venues were returned for each City

In [51]:
kerala_venues.groupby('City').count()

,City Latitude,City Longitude,Muncipality,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,,
Adoor,2,2,2,2,2,2,2
Aluva,4,4,4,4,4,4,4
Angamaly,5,5,5,5,5,5,5
Anthoor,4,4,4,4,4,4,4
Attingal,8,8,8,8,8,8,8
Chalakudy,4,4,4,4,4,4,4
Cherpulassery,5,5,5,5,5,5,5
Cherthala,6,6,6,6,6,6,6
Chittur-Thathamangalam,5,5,5,5,5,5,5


#### Let's find out how many unique categories can be curated from all the returned venues

In [12]:
print('There are {} uniques categories.'.format(len(kerala_venues['Venue Category'].unique())))

There are 95 uniques categories.


In [113]:
#get the unique Venue Categories from the list we generated above
kerala_unique_venues_df = pd.DataFrame({'Venue Category': kerala_venues['Venue Category'].unique()})
#sort in ascending value
kerala_unique_venues_df = kerala_venues_df.sort_values('Venue Category',ascending=True).reset_index(drop=True)
kerala_unique_venues_df.head()

,Venue Category
0,ATM
1,Accessories Store
2,Airport
3,Airport Terminal
4,Arcade


#### Form a weightage matrix, using below logic applied on venues


1.   Venue related to water body - 10
2.   Venue is must have for a residential area - 5
3.   Venue is good to have for a residential area - 2
4.   Any other venue - 1

This weightage matrix is formed, so that we can multiply this matrix with each city venue details, and will get weighted dataset based on the preference of venues we are looking for this project

Below weightage matrix is formed with initial undertsnading and unique list of categories. So please spare some time at this point to review the list and add new entries





In [134]:
weightage_array = [ ('ATM',5),('Accessories Store',1),('Airport',1),('Airport Terminal',1),('Arcade',1),('Arts & Crafts Store',1),('Asian Restaurant',1),
('BBQ Joint',1),('Bakery',2),('Bar',1),('Beach',10),('Bed & Breakfast',1),('Boutique',5),('Bowling Alley',1),('Burger Joint',1),('Bus Line',2),
('Bus Station',2),('Café',2),('Campground',1),('Chinese Restaurant',1),('Clothing Store',1),('Coffee Shop',1),('Comfort Food Restaurant',1),
('Concert Hall',1),('Convenience Store',5),('Cosmetics Shop',2),('Currency Exchange',1),('Department Store',5),('Diner',1),('Electronics Store',2),
('Falafel Restaurant',1),('Fast Food Restaurant',1),('Fish & Chips Shop',1),('Fish Market',5),('Flea Market',2),('Food',1),('Food & Drink Shop',1),
('Food Court',1),('Football Stadium',1),('Forest',1),('Fried Chicken Joint',1),('Furniture / Home Store',2),('General Travel',1),('Gift Shop',1),
('Grocery Store',5),('Gym',2),('Hotel',1),('Ice Cream Shop',1),('Indian Restaurant',1),('Indie Movie Theater',2),('Jewelry Store',1),('Juice Bar',1),
('Kerala Restaurant',1),('Lake',10),('Light Rail Station',2),('Liquor Store',1),('Lounge',1),('Market',5),('Mattress Store',1),('Men\'s Store',1),
('Middle Eastern Restaurant',1),('Mobile Phone Shop',2),('Motel',1),('Movie Theater',5),('Multicuisine Indian Restaurant',1),('Multiplex',5),
('Music Store',1),('Other Great Outdoors',1),('Park',5),('Pharmacy',5),('Pizza Place',1),('Platform',1),('Pub',1),('Resort',1),('Restaurant',1),
('River',10),('Sandwich Place',1),('Scenic Lookout',1),('Shopping Mall',2),('Shopping Plaza',1),('Snack Place',1),('Southern / Soul Food Restaurant',1),
('Spa',2),('Sporting Goods Shop',2),('Sports Club',1),('Stadium',1),('Student Center',1),('Supermarket',5),('Tea Room',1),('Temple',5),('Track',1),
('Track Stadium',1),('Train Station',2),('Travel & Transport',2),('Vegetarian / Vegan Restaurant',1) ]
weightage_matrix = pd.DataFrame(weightage_array, columns = ['Venue Category', 'Weight']) 
weightage_matrix

,Venue Category,Weight
0,ATM,5
1,Accessories Store,1
2,Airport,1
3,Airport Terminal,1
4,Arcade,1
5,Arts & Crafts Store,1
6,Asian Restaurant,1
7,BBQ Joint,1
8,Bakery,2
9,Bar,1


####Verify the weightage matrix against the unique venues list to find out missing data in weightage matrix

In the result set, look for any rows with weightage as NaN. Those are the venues, not available in weightage matrix. In that case, you need to update the weightage array in above cell with new Venue Category and corresponding weight. Then need to rerun from that cell onwards.

In [297]:
#to ensure duplicate columns are not added as part of reexecuting below step multiple times
kerala_unique_venues_df.drop(kerala_unique_venues_df.columns.difference(['Venue Category']), 1, inplace=True)
#merging df containing unique kerala venues and df containing venue weightages, to verify whether all venues are weighted
kerala_unique_venues_df = kerala_unique_venues_df.merge(weightage_matrix, left_on='Venue Category', right_on='Venue Category', how ='outer')
kerala_unique_venues_df.head()

,Venue Category,Weight
0,ATM,5
1,Accessories Store,1
2,Airport,1
3,Airport Terminal,1
4,Arcade,1


Analyze each neighbourhood

In [13]:
# one hot encoding
kerala_onehot = pd.get_dummies(kerala_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kerala_onehot['City'] = kerala_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [kerala_onehot.columns[-1]] + list(kerala_onehot.columns[:-1])
kerala_onehot = kerala_onehot[fixed_columns]

kerala_onehot.head()

,City,ATM,Accessories Store,Airport,Airport Terminal,Arcade,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Boutique,Bowling Alley,Burger Joint,Bus Line,Bus Station,Café,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Football Stadium,Forest,Fried Chicken Joint,Furniture / Home Store,General Travel,Gift Shop,Grocery Store,Gym,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Men's Store,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Store,Other Great Outdoors,Park,Pharmacy,Pizza Place,Platform,Pub,Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Shopping Mall,Shopping Plaza,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Club,Stadium,Student Center,Supermarket,Tea Room,Temple,Track,Track Stadium,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant
0,"Trivandrum, TVM",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Trivandrum, TVM",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Trivandrum, TVM",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Trivandrum, TVM",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Trivandrum, TVM",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by city and by taking the mean of the frequency of occurrence of each category

In [14]:
kerala_grouped = kerala_onehot.groupby('City').mean().reset_index()
kerala_grouped

,City,ATM,Accessories Store,Airport,Airport Terminal,Arcade,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Boutique,Bowling Alley,Burger Joint,Bus Line,Bus Station,Café,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Football Stadium,Forest,Fried Chicken Joint,Furniture / Home Store,General Travel,Gift Shop,Grocery Store,Gym,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Men's Store,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Store,Other Great Outdoors,Park,Pharmacy,Pizza Place,Platform,Pub,Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Shopping Mall,Shopping Plaza,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Club,Stadium,Student Center,Supermarket,Tea Room,Temple,Track,Track Stadium,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant
0,Adoor,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.5,0.00,0.00,0.000,0.000000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Aluva,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000,0.000000,0.250000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,Angamaly,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000,0.000000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.200000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.400000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Anthoor,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000,0.000000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00

Apply the weightage matrix at this stage by multiplying the weightage vector with mean of dummy values calculated. The result will be a new dataframe *kerala_grouped_weighted* which we can use in clustering

In [253]:
kerala_grouped_weighted = pd.concat([kerala_grouped.iloc[: , [0]], kerala_grouped.iloc[:, 1:(kerala_unique_venues_df.shape[0]+1)] * weightage_matrix['Weight'].values], axis=1)

Just to re verify the multiplication worked, we can divide the "Movie Theater" weighted value with initial value. The result should be an array of 5, which is the weight we allocated for Movie Theater

In [252]:
kerala_grouped_weighted['Movie Theater'][kerala_grouped_weighted['Movie Theater'] > 0] / kerala_grouped['Movie Theater'][kerala_grouped['Movie Theater'] > 0]

26    5.0
31    5.0
32    5.0
37    5.0
38    5.0
41    5.0
42    5.0
47    5.0
48    5.0
55    5.0
56    5.0
59    5.0
Name: Movie Theater, dtype: float64

###Let's create the new dataframe and display the top 10 venues for each neighborhood.

Function for returning the top n venues

In [254]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now we can extract the top ten venues of each city and create a new dataframe, which can then be used for analysis

In [255]:
#top n number of venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = kerala_grouped_weighted['City']

for ind in np.arange(kerala_grouped_weighted.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kerala_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adoor,Concert Hall,Chinese Restaurant,Vegetarian / Vegan Restaurant,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant,Fast Food Restaurant
1,Aluva,Hotel,Diner,Concert Hall,Convenience Store,Cosmetics Shop,Currency Exchange,Department Store,Electronics Store,Falafel Restaurant,Fast Food Restaurant
2,Angamaly,Multiplex,Hotel,Indian Restaurant,Bus Station,Flea Market,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant
3,Anthoor,Indian Restaurant,Kerala Restaurant,Café,Bus Station,Vegetarian / Vegan Restaurant,Fish Market,Currency Exchange,Department Store,Diner,Electronics Store
4,Attingal,Bus Station,Bakery,Indian Restaurant,Chinese Restaurant,Mobile Phone Shop,Vegetarian / Vegan Restaurant,Fish Market,Department Store,Diner,Electronics Store
5,Chalakudy,Bus Station,Bakery,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Flea Market,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant
6,Cherpulassery,Currency Exchange,Restaurant,BBQ Joint,Bus Station,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Flea Market,Department Store,Diner,Electronics Store
7,Cherthala,Restaurant,Pub,Resort,Flea Market,Spa,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Convenience Store,Cosmetics Shop,Currency Exchange
8,Chittur-Thathamangalam,Vegetarian / Vegan Restaurant,Fish Market,Indian Restaurant,Tea Room,Bus Station,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store
9,Eloor,Scenic Lookout,Market,Vegetarian / Vegan Restaurant,Fish Market,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant


## . Cluster Neighborhoods

Let's cluster with 5 clusters for now to validate the dataset we prepared

In [256]:
# set number of clusters
kclusters = 5

kerala_grouped_clustering = kerala_grouped_weighted.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kerala_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 1, 0, 0, 0, 0, 3, 0, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [257]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

kerala_merged = kerala_city_filtered_df

# merge kerala_merged with kerala_city_filtered_df to add latitude/longitude for each city having atleast one venue
kerala_merged = kerala_merged.join(city_venues_sorted.set_index('City'), on='City')

kerala_merged.head() # check the last columns!

,City,Muncipality,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Trivandrum, TVM",Trivandrum,8.487803,76.951847,3,Indian Restaurant,Hotel,Clothing Store,Multiplex,Movie Theater,Indie Movie Theater,Shopping Mall,Department Store,Restaurant,Food & Drink Shop
1,Kozhikode,Kozhikode,11.258608,75.778874,3,Indian Restaurant,Clothing Store,Restaurant,Coffee Shop,Hotel,Movie Theater,Vegetarian / Vegan Restaurant,Beach,Café,Fast Food Restaurant
2,Kochi,Ernakulam,9.931308,76.267414,3,Restaurant,Bakery,Gym,Airport,Fried Chicken Joint,Food Court,Fish Market,Currency Exchange,Department Store,Diner
3,Kollam,Kollam,8.887951,76.595501,2,Movie Theater,Bakery,Clothing Store,Indie Movie Theater,Coffee Shop,Department Store,Market,Bus Station,Fast Food Restaurant,Beach
4,Thrissur,Thrissur,10.525626,76.213254,3,Indian Restaurant,Movie Theater,Multiplex,Clothing Store,Bus Station,Vegetarian / Vegan Restaurant,Shopping Mall,Food,Mattress Store,Chinese Restaurant


Finally, let's visualize the resulting clusters

In [258]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kerala_merged['City Latitude'], kerala_merged['City Longitude'], kerala_merged['City'], kerala_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##Examine Clusters

#### Cluster 1

In [259]:
kerala_merged.loc[kerala_merged['Cluster Labels'] == 0, kerala_merged.columns[[0,1] + list(range(5, kerala_merged.shape[1]))]]

,City,Muncipality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Manjeri,Malappuram,Indian Restaurant,Juice Bar,Fried Chicken Joint,Bus Station,Vegetarian / Vegan Restaurant,Fish Market,Currency Exchange,Department Store,Diner,Electronics Store
11,Payyanur,Kannur,Indian Restaurant,Bus Station,Shopping Mall,Vegetarian / Vegan Restaurant,Fish Market,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store
21,Wadakkancherry,Thrissur,ATM,Currency Exchange,Asian Restaurant,Bus Station,Flea Market,Department Store,Diner,Electronics Store,Falafel Restaurant,Fast Food Restaurant
24,Kondotty,Malappuram,Airport Terminal,Café,Bus Station,Liquor Store,Vegetarian / Vegan Restaurant,Flea Market,Department Store,Diner,Electronics Store,Falafel Restaurant
26,Feroke,Kozhikode,Train Station,Fried Chicken Joint,Restaurant,Bus Station,Vegetarian / Vegan Restaurant,Fish & Chips Shop,Cosmetics Shop,Currency Exchange,Department Store,Diner
28,Ettumanoor,Kottayam,Flea Market,Indian Restaurant,Coffee Shop,Bus Station,Vegetarian / Vegan Restaurant,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant
30,Chalakudy,Thrissur,Bus Station,Bakery,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Flea Market,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant
31,Payyoli,Kozhikode,Indian Restaurant,ATM,Train Station,Bus Station,Fish Market,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store
33,Kottakkal,Malappuram,Coffee Shop,Juice Bar,Asian Restaurant,Restaurant,Bus Station,Vegetarian / Vegan Restaurant,Fish Market,Currency Exchange,Department Store,Diner
40,Kattappana,Idukki,Restaurant,Lake,Bus Station,Shopping Plaza,Vegetarian / Vegan Restaurant,Fish & Chips Shop,Cosmetics Shop,Currency Exchange,Department Store,Diner


Upon analyzing Cluster 1 and neighbourhood venues, I would like to segment this as a "**Place to Eat + Transportation Centre**"

#### Cluster 2

In [260]:
kerala_merged.loc[kerala_merged['Cluster Labels'] == 1, kerala_merged.columns[[0,1] + list(range(5, kerala_merged.shape[1]))]]

,City,Muncipality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Kanhangad,Kasaragod,Multiplex,Ice Cream Shop,Hotel,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant,Fast Food Restaurant
49,Angamaly,Ernakulam,Multiplex,Hotel,Indian Restaurant,Bus Station,Flea Market,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant


Upon analyzing Cluster 2 and neighbourhood venues, I would like to segment this as a "**Theaters + Place to Eat**"

#### Cluster 3

In [261]:
kerala_merged.loc[kerala_merged['Cluster Labels'] == 2, kerala_merged.columns[[0,1] + list(range(5, kerala_merged.shape[1]))]]

,City,Muncipality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Kollam,Kollam,Movie Theater,Bakery,Clothing Store,Indie Movie Theater,Coffee Shop,Department Store,Market,Bus Station,Fast Food Restaurant,Beach
8,Thrippunithura,Ernakulam,Train Station,Movie Theater,Electronics Store,Bakery,Vegetarian / Vegan Restaurant,Fish Market,Cosmetics Shop,Currency Exchange,Department Store,Diner
12,Koyilandy,Kozhikode,Train Station,Movie Theater,Football Stadium,Bus Station,Vegetarian / Vegan Restaurant,Fish & Chips Shop,Cosmetics Shop,Currency Exchange,Department Store,Diner
23,Nedumangad,Trivandrum,Indie Movie Theater,Hotel,Flea Market,Movie Theater,Bus Station,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant
41,Mukkam,Kozhikode,Convenience Store,Cosmetics Shop,Restaurant,Movie Theater,Concert Hall,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant
57,Piravom,Ernakulam,Asian Restaurant,Ice Cream Shop,Music Store,Movie Theater,Bus Station,Fish & Chips Shop,Fish Market,Currency Exchange,Department Store,Diner
59,Perumbavoor,Ernakulam,Gym,Department Store,Chinese Restaurant,Fried Chicken Joint,Movie Theater,Food Court,Bus Line,Fish & Chips Shop,Currency Exchange,Diner
62,Pala,Kottayam,Movie Theater,Bus Station,Stadium,Bakery,Bar,Vegetarian / Vegan Restaurant,Fish Market,Currency Exchange,Department Store,Diner


Upon analyzing Cluster 3 and neighbourhood venues, I would like to segment this as a "**Potential Residential Area** because of the presence of venues such as Department Store, Fish Market, Bakery, Bus Station, Train Station, etc.

Need to further check the places in this segment having water body presence. Which we will do below

In [299]:
Cluster_2_cities = kerala_merged.loc[kerala_merged['Cluster Labels'] == 2]['City']
Cluster_2_full_venues = kerala_grouped_weighted[kerala_grouped_weighted.City.isin(Cluster_2_cities)].reset_index(drop=True)
Cluster_2_full_venues[['City', 'Beach', 'Lake', 'River']]

,City,Beach,Lake,River
0,Kollam,0.47619,0.0,0.0
1,Koyilandy,0.00000,0.0,0.0
2,Mukkam,0.00000,0.0,0.0
3,Nedumangad,0.00000,0.0,0.0
4,Pala,0.00000,0.0,0.0
5,Perumbavoor,0.00000,0.0,0.0
6,Piravom,0.00000,0.0,0.0
7,Thrippunithura,0.00000,0.0,0.0


From the above, City **Kollam** is a contender for the project location

#### Cluster 4

In [262]:
kerala_merged.loc[kerala_merged['Cluster Labels'] == 3, kerala_merged.columns[[0,1] + list(range(5, kerala_merged.shape[1]))]]

,City,Muncipality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Trivandrum, TVM",Trivandrum,Indian Restaurant,Hotel,Clothing Store,Multiplex,Movie Theater,Indie Movie Theater,Shopping Mall,Department Store,Restaurant,Food & Drink Shop
1,Kozhikode,Kozhikode,Indian Restaurant,Clothing Store,Restaurant,Coffee Shop,Hotel,Movie Theater,Vegetarian / Vegan Restaurant,Beach,Café,Fast Food Restaurant
2,Kochi,Ernakulam,Restaurant,Bakery,Gym,Airport,Fried Chicken Joint,Food Court,Fish Market,Currency Exchange,Department Store,Diner
4,Thrissur,Thrissur,Indian Restaurant,Movie Theater,Multiplex,Clothing Store,Bus Station,Vegetarian / Vegan Restaurant,Shopping Mall,Food,Mattress Store,Chinese Restaurant
5,Kannur,Kannur,Shopping Mall,Indian Restaurant,Fast Food Restaurant,Train Station,Platform,Kerala Restaurant,Bakery,Restaurant,Vegetarian / Vegan Restaurant,Cosmetics Shop
6,Palakkad,Palakkad,Hotel,Indian Restaurant,Vegetarian / Vegan Restaurant,Light Rail Station,Movie Theater,Coffee Shop,Clothing Store,Bus Station,Shopping Mall,Other Great Outdoors
9,Vatakara,Kozhikode,Travel & Transport,Train Station,Bus Station,Fast Food Restaurant,Bakery,Flea Market,Currency Exchange,Department Store,Diner,Electronics Store
13,Kodungallur,Thrissur,Café,Supermarket,Sports Club,Bakery,Vegetarian / Vegan Restaurant,Fish Market,Cosmetics Shop,Currency Exchange,Department Store,Diner
15,Kalamassery,Ernakulam,Indian Restaurant,Sporting Goods Shop,Vegetarian / Vegan Restaurant,Flea Market,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant
16,Neyyattinkara,Trivandrum,Train Station,Chinese Restaurant,Stadium,Bakery,Vegetarian / Vegan Restaurant,Fish Market,Cosmetics Shop,Currency Exchange,Department Store,Diner


Upon analyzing Cluster 4 and neighbourhood venues, I would like to segment this as a "**City Centre or Business Area** because of the presence of lots of business venues

So we are not considering this for for the project location

#### Cluster 5

In [263]:
kerala_merged.loc[kerala_merged['Cluster Labels'] == 4, kerala_merged.columns[[0,1] + list(range(5, kerala_merged.shape[1]))]]

,City,Muncipality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Parappanangadi,Malappuram,Train Station,Vegetarian / Vegan Restaurant,Flea Market,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant,Fast Food Restaurant
22,Irinjalakuda,Thrissur,Train Station,Vegetarian / Vegan Restaurant,Flea Market,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant,Fast Food Restaurant
34,Karunagappalli,Kollam,Train Station,Platform,Indie Movie Theater,Vegetarian / Vegan Restaurant,Fish Market,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store
44,Nileshwaram,Kasaragod,Train Station,Department Store,Vegetarian / Vegan Restaurant,Flea Market,Cosmetics Shop,Currency Exchange,Diner,Electronics Store,Falafel Restaurant,Fast Food Restaurant
55,Kottarakara,Kollam,Train Station,Indie Movie Theater,Motel,Vegetarian / Vegan Restaurant,Fish Market,Cosmetics Shop,Currency Exchange,Department Store,Diner,Electronics Store


Upon analyzing Cluster 5 and neighbourhood venues, I would like to segment this as a "**Transportation Centre + Potential Residential Area** .

Need to further check the places in this segment having water body presence. Which we will do below

In [301]:
Cluster_5_cities = kerala_merged.loc[kerala_merged['Cluster Labels'] == 4]['City']
Cluster_5_full_venues = kerala_grouped_weighted[kerala_grouped_weighted.City.isin(Cluster_5_cities)].reset_index(drop=True)
Cluster_5_full_venues[['City', 'Beach', 'Lake', 'River']]

,City,Beach,Lake,River
0,Irinjalakuda,0.0,0.0,0.0
1,Karunagappalli,0.0,0.0,0.0
2,Kottarakara,0.0,0.0,0.0
3,Nileshwaram,0.0,0.0,0.0
4,Parappanangadi,0.0,0.0,0.0


Unfortunately, no cities in this segment has a water body nearby and hence we are skipping this cluster.